In [91]:
SEASON = '1976-77'
LEAGUE = 'NBA'

In [92]:
import pandas as pd
import myconstants as c
from scipy.stats import zscore

NUM_GAMES = c.NUM_GAMES[SEASON]
MIN_GAMES = c.MIN_GAMES[SEASON]

p_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_{LEAGUE}_Player_PerGame.csv')
t_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_{LEAGUE}_Team_Standings.csv')
p_advanced_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_{LEAGUE}_Player_Advanced.csv')


# join player per game and advanced stats
p_advanced_df = p_advanced_df.drop(columns=['Player', 'Tms', 'Pos', 'Age', 'G', 'MP'])
p_df = p_df.merge(p_advanced_df, on='PlayerID')

# populate team df with team codes
t_df['TeamCode'] = t_df['Team'].apply(lambda x: c.TEAM_CODE[x])

# average team wins per player
def average_wins(tms):
    tms_list = tms.split(',')
    tms_wins = []
    for t in tms_list:
        tms_wins.append(t_df.loc[t_df['TeamCode'] == t]['OverallW'].values[0])
    return sum(tms_wins) / len(tms_wins)
p_df['TmsWins'] = p_df['Tms'].apply(average_wins)

# players who haven't played enough games don't qualify
p_df = p_df.loc[p_df['G'] >= MIN_GAMES]

# ranking algorithm
p_df['zTotals'] = 1.5*zscore(p_df['PTS'] + p_df['TRB'] + p_df['AST'] + p_df['STL'] + p_df['BLK'])
p_df['zWS'] = zscore(p_df['WS'])
p_df['zVORP'] = zscore(p_df['VORP'])
p_df['zBPM'] = zscore(p_df['BPM'])
p_df['zTmsWins'] = zscore(p_df['TmsWins'])
p_df['RawScore'] = p_df['zTotals'] + p_df['zWS'] + p_df['zVORP'] + p_df['zBPM'] + p_df['zTmsWins']
p_df['PlayerScore'] = p_df['RawScore'] + c.PLAYER_SCORE_ADDITION # more positive PlayerScores
p_df['PlayerScore'] = p_df['PlayerScore'].apply(lambda x: pow(x, c.PLAYER_SCORE_EXPONENT) if x > 1 else x) # make higher PlayerScores more significant
p_df = p_df.sort_values('PlayerScore', ascending=False)

# Move [PlayerScore] to the right of [Pos], drop PlayerID, index rows starting from 1, stylistic choice
col = p_df.pop('PlayerScore')
p_df.insert(p_df.columns.get_loc('Pos') + 1, col.name, col)
p_df = p_df.drop('PlayerID', axis=1)
p_df = p_df.reset_index(drop=True)
p_df.index = p_df.index + 1

p_df.head(50)

,Player,Tms,Pos,PlayerScore,Age,G,GS,MP,FG,FGA,...,DBPM,BPM,VORP,TmsWins,zTotals,zWS,zVORP,zBPM,zTmsWins,RawScore
1,Kareem Abdul-Jabbar,LAL,C,141.902487,29,82,NaN,36.8,10.8,18.7,...,2.6,9.4,8.7,53.0,4.694993,4.728433,5.798162,4.380550,1.603568,21.205706
2,Bill Walton,POR,C,82.597392,24,65,NaN,34.8,7.6,14.3,...,2.3,6.7,5.0,49.0,3.567294,2.142854,3.017191,3.173499,1.065241,12.966079
3,Bobby Jones,DEN,PF,77.693689,25,82,NaN,29.5,6.1,10.7,...,3.6,7.0,5.5,50.0,1.892330,2.415020,3.392998,3.307616,1.199823,12.207787
4,Bob Lanier,DET,C,75.267944,28,64,NaN,38.2,10.6,19.8,...,0.3,5.4,4.6,44.0,3.948722,2.176874,2.716546,2.592326,0.392331,11.826799
5,Julius Erving,PHI,SF,74.859931,26,82,77.0,35.9,8.4,16.7,...,1.3,4.5,4.8,50.0,2.920526,2.585124,2.866869,2.189976,1.199823,11.762317
6,Elvin Hayes,WSB,PF,71.378000,31,82,NaN,41.0,9.3,18.5,...,0.8,2.8,4.1,48.0,3.716548,2.789249,2.340739,1.429981,0.930659,11.207175
7,Artis Gilmore,CHI,C,68.962736,27,82,NaN,35.1,7.0,13.3,...,1.3,4.2,4.5,44.0,2.903942,2.823269,2.641385,2.055859,0.392331,10.816786
8,David Thompson,DEN,SG,60.304165,22,82,NaN,36.6,10.0,19.8,...,-0.6,2.5,3.4,50.0,2.754688,2.312958,1.814610,1.295864,1.199823,9.377942
9,Dan Issel,DEN,C,58.302524,28,79,NaN,31.7,8.4,16.2,...,-0.4,2.7,3.0,50.0,2.555682,2.380999,1.513964,1.385275,1.199823,9.035743
10,George Gervin,SAS,SF,57.621536,24,82,NaN,33.0,8.9,16.3,...,-0.4,3.5,3.8,44.0,2.423012,2.244916,2.115255,1.742920,0.392331,8.918433
